In [11]:
import json
import os
import pandas as pd
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils

In [4]:
s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
f, _, _ = s3_access_utils.download_from_url('https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/jane/fish_id/bolaks_data_pairs.csv')
df = pd.read_csv(f)

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

query = '''
    SELECT * FROM (
      (SELECT * FROM prod.crop_annotation cas
      INNER JOIN prod.annotation_state pas on pas.id=cas.annotation_state_id
      WHERE cas.service_id = (SELECT ID FROM prod.service where name='BATI')
      AND cas.annotation_state_id = 3
      AND cas.pen_id=88) a
    RIGHT JOIN 
      (SELECT left_crop_url, estimated_weight_g, akpd_score FROM prod.biomass_computations
      WHERE prod.biomass_computations.captured_at between '2020-02-10' and '2020-03-10'
      AND prod.biomass_computations.akpd_score > 0.9) bc 
    ON 
      (a.left_crop_url=bc.left_crop_url)
    ) x
    WHERE x.captured_at between '2020-02-10' and '2020-03-10'
    AND x.pen_id = 88
    AND x.group_id = '88';
'''

df = rds_access_utils.extract_from_database(query)
df = df.sort_values('captured_at')
df = df[df.akpd_score > 0.99].copy(deep=True)
df.index = pd.to_datetime(df.captured_at)
df['hour'] = df.index.hour
